In [55]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

import pandas as pd
import json

In [56]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
# conexion a amigocloud
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [57]:
fecha = '2023-05-22'

In [58]:
query_dia = {'query': 'select id, canhero from dataset_289611 where date(fecha_registro)=\'{fech}\''.format(fech=fecha)}
project = 'https://app.amigocloud.com/api/v1/projects/31702/sql'
select_dia = amigocloud.get(project, query_dia)
data_dia = select_dia['data']
data_dia

[{'id': 25, 'canhero': '388 / AGROPECUARIA MARIANA S.R.L.'}]

In [59]:
id_insp = data_dia[0]['id']
id_insp

25

In [60]:
# consulta para buscar por id
query = {'query': 'select * from dataset_289611 where id={id}'.format(id=id_insp)}
project = 'https://app.amigocloud.com/api/v1/projects/31702/sql'
select = amigocloud.get(project, query)
data = select['data'][0]
# cambio del formato de fecha
data['fecha_registro'] = datetime.strptime(data['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
# convercion a objeto
insp = collections.namedtuple("insp", data.keys())(*data.values())
insp

insp(fecha_registro='22/05/2023', amigo_id='fdbf1104c44f4be0b0493907b36fb62e', canhero='388 / AGROPECUARIA MARIANA S.R.L.', lote='L30.8', area=7.48, textura='F', variedad='RB-2', repeticiones=None, filas_por_surco=3, origen_de_semilla='PROPIO', edad_de_semilla='HOJA', ubicacion='0101000020E6100000737275B636954FC0C1EE2A5A6F4831C0', obs='Siembra con tres cañas, tamaño del cañoto grande ', propiedad='19 / SANTA BARBARA', usuario='Rogelio Acuña Rodríguez', id=25)

In [61]:
# consulta para buscar 
query = {'query': 'select * from dataset_289610 where calidad_ref_muestra=\'{amigo_id}\''.format(amigo_id=insp.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/31702/sql'
select = amigocloud.get(project, query)
data_muestras = select['data']

In [62]:
data_muestras

[{'fecha_registro': '2023-05-22 17:33:25+00:00',
  'amigo_id': '96c65d87998c46c9ab00ec467325d559',
  'punto': '0101000020E61000005DEEDD301B954FC022C08EA4494831C0',
  'profundidad_surco': 16.0,
  'distancia_entre_surco': 1.5,
  'tamaño_de_cañoto': 60.0,
  'cantidad_tierra': 10.0,
  'falla_de_tapado': 38.0,
  'peso_en_5m': 9.5,
  'cantidad_de_semilla': 12.6666666666667,
  'calidad_ref_muestra': 'fdbf1104c44f4be0b0493907b36fb62e',
  'yemas_metro': 25.0,
  'yemas_no_viables': 0.0,
  'obs': 'Tamaño del cañoto ',
  'id': 38},
 {'fecha_registro': '2023-05-22 18:00:29+00:00',
  'amigo_id': '50f446dc41434412908a36cd1d1ab22f',
  'punto': '0101000020E6100000EA0203AD65954FC0ABA7A0EC774831C0',
  'profundidad_surco': 19.0,
  'distancia_entre_surco': 1.5,
  'tamaño_de_cañoto': 60.0,
  'cantidad_tierra': 10.0,
  'falla_de_tapado': 0.0,
  'peso_en_5m': 8.79,
  'cantidad_de_semilla': 11.72,
  'calidad_ref_muestra': 'fdbf1104c44f4be0b0493907b36fb62e',
  'yemas_metro': 22.0,
  'yemas_no_viables': 1.0,
  '

In [63]:
keys_to_avg = ['profundidad_surco', 
               'distancia_entre_surco', 
               'tamaño_de_cañoto', 
               'cantidad_tierra', 
               'falla_de_tapado', 
               'peso_en_5m', 
               'cantidad_de_semilla', 
               'yemas_metro', 
               'yemas_no_viables']
averages = {}
for key in keys_to_avg:
    total = sum(d[key] for d in data_muestras)
    averages[key] = total / len(data_muestras)

averages['yemas_metro'] = int(averages['yemas_metro'])

In [64]:
averages

{'profundidad_surco': 17.5,
 'distancia_entre_surco': 1.5,
 'tamaño_de_cañoto': 60.0,
 'cantidad_tierra': 10.0,
 'falla_de_tapado': 19.0,
 'peso_en_5m': 9.145,
 'cantidad_de_semilla': 12.19333333333335,
 'yemas_metro': 23,
 'yemas_no_viables': 0.5}

In [65]:
data = collections.namedtuple("data", averages.keys())(*averages.values())

In [66]:
data

data(profundidad_surco=17.5, distancia_entre_surco=1.5, tamaño_de_cañoto=60.0, cantidad_tierra=10.0, falla_de_tapado=19.0, peso_en_5m=9.145, cantidad_de_semilla=12.19333333333335, yemas_metro=23, yemas_no_viables=0.5)

In [67]:
query_fotos = {'query': 'select insp.lote, muestra.obs, gal.s3_filename from dataset_289611 insp inner join dataset_289610 muestra on insp.amigo_id=muestra.calidad_ref_muestra inner join gallery_41234 gal on muestra.amigo_id=gal.source_amigo_id where insp.amigo_id=\'{amigo_id}\''.format(amigo_id=insp.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/31702/sql'
select_fotos = amigocloud.get(project, query_fotos)
fotos = select_fotos['data']

In [68]:
fotos

[{'lote': 'L30.8',
  'obs': 'Tamaño del cañoto ',
  's3_filename': '20230522_134219.jpg'},
 {'lote': 'L30.8',
  'obs': 'Tamaño del cañoto ',
  's3_filename': '20230522_134848.jpg'}]

In [69]:
doc = DocxTemplate(ruta + "/templates/tpl_rpt_calidad-siembra.docx")

In [70]:
# SE DESCARGAN LAS FOTOS
lista_fotos_inline = []
i = 1
for foto in fotos:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({
        'foto': docxtpl.InlineImage(doc, image_descriptor=ruta+'/fotos/' + foto['s3_filename'], width=Mm(60)), 
        'muestra': 'M' + str(i),
        'obs':foto['obs']
    })
    i=i+1

In [71]:
lista_fotos_inline[0]['muestra']

'M1'

In [72]:
firma_respon = None
if insp.usuario == 'Rogelio Acuña Rodríguez':
    firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_rogelio.png', width=Mm(60))
else:
    firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_jaldin.png', width=Mm(60))

In [73]:
firma_respon

In [74]:
# NORMBRE DEL ARCHIVO
filename = '/_' + insp.canhero.split(' / ')[0] + '_CDS_' + insp.fecha_registro.replace('/','-') + '_' + insp.canhero.split(' / ')[1] + '_' + str(insp.id) + '.docx'
filename

'/_388_CDS_22-05-2023_AGROPECUARIA MARIANA S.R.L._25.docx'

In [75]:
# CREAR EL CONTEXTO DEL DOC Y CREAR EL DOC
context = {'insp':insp, 'data':data, 'fotos':lista_fotos_inline, 'responsable':firma_respon}
doc.render(context)
doc.save(ruta + '/informes' + filename)
print('Trarea terminada.')

Trarea terminada.
